In [3]:
!pip install selenium
!pip install webdriver-manager
!pip install nltk

In [4]:
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
lista = []

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Wallace\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [21]:
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(service=service)

try:
    driver.get("https://www.imdb.com/title/tt1677720/reviews/?ref_=nv_sr_srsg_0_tt_8_nm_0_in_0_q_playe")
    time.sleep(5)

    while True:
        try:
            mais25_button = driver.find_element(
                By.XPATH,
                ".//button[contains(@class, 'ipc-see-more__button')]"
            )
            driver.execute_script("arguments[0].scrollIntoView();", mais25_button)
            time.sleep(2)
            mais25_button.click()
            time.sleep(5)
        except:
            break

    spoiler_buttons = driver.find_elements(
        By.XPATH,
        ".//button[contains(@class, 'review-spoiler-button')]"
    )
    for button in spoiler_buttons:
        try:
            driver.execute_script("arguments[0].scrollIntoView();", button)
            time.sleep(1)
            button.click()
            time.sleep(1)
        except:
            print("Erro ao clicar no botão de spoiler.")

    html = driver.page_source
    pagina = bs(html, 'html.parser')

    for x in pagina.find_all(class_='ipc-list-card__content'):
        titulo = x.find(class_="ipc-title__text").text if x.find(class_="ipc-title__text") else "Sem título"
        desc = x.find(class_="ipc-html-content-inner-div").text if x.find(class_="ipc-html-content-inner-div") else "Sem descrição"
        lista.append({'titulo': titulo, 'descricao': desc})
        
finally:
    driver.quit()


In [23]:
df = pd.DataFrame(lista)
display(df)

,titulo,descricao
0,Not a bad flick,"I'll go with 5 on this one, I don't think it's..."
1,What a stinker,I rented this movie knowing nothing about it. ...
2,Sexist piece of crap!,Absolutely one of the worst movies I've seen i...
3,I do not want to see this movie again,"One of the weakest movies, poorly played. The ..."
4,On some roads it is better not to fold ... An...,On some roads it is better not to fold ...And ...
...,...,...
1972,A Nerd's Nerd Film,Ready Player One is quite possibly the biggest...
1973,You don't have to have read the book to find t...,"Sat through this dull, predictable, kids movie..."
1974,Spielberg Returns to the Blockbuster in This T...,The King of the Blockbuster is back! In the de...
1975,Absolutely Fantastic,Geeks rejoice. Spielberg is back to form!! I w...


In [25]:
sia = SentimentIntensityAnalyzer()

if 'descricao' not in df.columns or df.empty:
    print("Erro: O DataFrame está vazio ou a coluna 'descricao' não existe.")
else:
    def analisar_sentimento(texto):
        if texto:  
            sentimento = sia.polarity_scores(texto)
            return sentimento['compound'] 
        return None

    df['sentimento'] = df['descricao'].apply(analisar_sentimento)

    def classificar_sentimento(score):
        if score is None:
            return "Sem análise"
        elif score > 0.05:
            return "Positivo"
        elif score < -0.05:
            return "Negativo"
        else:
            return "Neutro"

    df['classificacao'] = df['sentimento'].apply(classificar_sentimento)

    display(df.head()) 
    print("\nDistribuição de classificações:")
    print(df['classificacao'].value_counts())

,titulo,descricao,sentimento,classificacao
0,Not a bad flick,"I'll go with 5 on this one, I don't think it's...",-0.2695,Negativo
1,What a stinker,I rented this movie knowing nothing about it. ...,-0.9662,Negativo
2,Sexist piece of crap!,Absolutely one of the worst movies I've seen i...,-0.9821,Negativo
3,I do not want to see this movie again,"One of the weakest movies, poorly played. The ...",-0.1901,Negativo
4,On some roads it is better not to fold ... An...,On some roads it is better not to fold ...And ...,0.9945,Positivo



Distribuição de classificações:
classificacao
Positivo    1626
Negativo     316
Neutro        35
Name: count, dtype: int64
